In [1]:
from dataset.Dataset import TGS_Dataset
from model.Models import UnetSeNext50
from contextlib import contextmanager
import time
import os

In [2]:
@contextmanager
def timer(title):
    t0 = time.time()
    yield
    print("{} - done in {:.0f}s".format(title, time.time() - t0))

In [3]:
TRAIN_PATH = './train/'
# AUX_PATH = './Data/auxiliary_data'
LOAD_PATHS = None
# LOAD_PATHS = ['./Saves7/UnetSeNext50/2018-10-05 01:49_Fold1_Epoach54_reset0_val0.877',
#              './Saves3/UnetSeNext50/2018-10-02 18:17_Fold2_Epoach36_reset0_val0.878',
#              './Saves5/UnetSeNext50/2018-10-03 19:38_Fold3_Epoach49_reset0_val0.857',
#              './Saves3/UnetSeNext50/2018-10-02 23:43_Fold4_Epoach46_reset0_val0.863',
#              './Saves/UnetSeNext50/2018-10-02 00:22_Fold5_Epoach135_reset1_val0.872']

In [4]:
##############################
LOSS = 'lovasz'
OPTIMIZER = 'SGD'
PRETRAINED = True
N_EPOCH = 120   ## 120 or 140 is better, 30 just for debug
BATCH_SIZE = 32
NET = UnetSeNext50
###########OPTIMIZER###########
LR = 1e-2
T_MAX = 60    ## 60 or 70 is better, 15 just for debug
T_MUL = 1
LR_MIN = 0

In [6]:
train_dataset = TGS_Dataset(TRAIN_PATH)
loaders, ids = train_dataset.yield_dataloader(num_workers=11, batch_size=BATCH_SIZE, nfold=5)

1562 empty masks out of 4000 total masks


In [ ]:
for i, (train_loader, val_loader) in enumerate(loaders, 1):
    with timer('Fold {}'.format(i)):
        net = NET(lr=LR, fold=i)
        net.define_criterion(LOSS)
        net.optmizer_and_scheduler(T_max=T_MAX, T_mul=T_MUL, lr_min=LR_MIN)
    
        if LOAD_PATHS is not None:
            if LOAD_PATHS[i - 1] is not None:
                net.load_model(LOAD_PATHS[i - 1])

        net.train_and_valid(train_loader, val_loader, n_epoch=N_EPOCH)